Laboratorio #7
- Marco Ramírez
- Estuardo Hernández

Ataque de Extracción

In [2]:
# Importamos las librerías a utilizar
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, MaxPooling2D
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from art.estimators.classification import KerasClassifier #No soporta TF 2
from art.attacks.evasion import FastGradientMethod
from art.utils import load_dataset

# Disabling eager execution from TF 2
tf.compat.v1.disable_eager_execution()

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

In [3]:
# Utilizamos las 939 imágenes que separamos para el ataque
# Dimensiones de cada imagen
image_size=(80,80)
# Tamaño del lote de datos por cada iteración
batch_size = 32
# Aplicar inversión horizontal a la imagen
datagen_train=ImageDataGenerator(horizontal_flip=True)
# Crear el generador de datos de entrenamiento con los parámetros definidos
train_generator=datagen_train.flow_from_directory("./random_imgs/",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode ="rgb",
    shuffle=True,
    subset='training'
) # Este subset permite obtener un subconjunto de los datos de entrenamiento, en este caso se obtiene el 70% de los datos para entrenamiento

# Aplicar inversión horizontal a la imagen
datagen_test=ImageDataGenerator(horizontal_flip=True)
# Crear el generador de datos de pruebas con los parámetros definidos
test_generator=datagen_test.flow_from_directory("./random_imgs/",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode ="rgb",
    shuffle=True,
    subset='training'
) # Este subset permite obtener un subconjunto de los datos de entrenamiento, en este caso se obtiene el 30% de los datos para pruebas

Found 939 images belonging to 25 classes.
Found 939 images belonging to 25 classes.


In [4]:
#Cargamos el modelo
vulnerable_model = tf.keras.models.load_model("malware-class.h5")

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# Utilizamos el KerasClassifier
clasificador = KerasClassifier(
    model=vulnerable_model,
    clip_values=(0, 255))

In [6]:
from art.attacks.extraction import CopycatCNN
# Creating the "neural net thief" object
# that will steal the original classifier
copycat_cnn = CopycatCNN(
    batch_size_fit=256,
    batch_size_query=256,
    nb_epochs=20,
    nb_stolen=20000,
    classifier=clasificador
    )

In [7]:
#Definimos un nuevo modelo
def create_blank_model():
    CNNmodel = Sequential()
    CNNmodel.add(Conv2D(8, (3, 3), activation='relu', padding='same', input_shape=(80, 80, 3)))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(Conv2D(8, (3, 3), activation='relu',  padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(MaxPooling2D((2, 2)))
    # Más capas con distintos filtros
    CNNmodel.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(Conv2D(16, (3, 3), activation='relu',  padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(MaxPooling2D((2, 2)))
    CNNmodel.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(MaxPooling2D((2, 2)))
    CNNmodel.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(MaxPooling2D((2, 2)))
    # Agregar capa para reducir el overfitting durante el entrenamiento
    CNNmodel.add(Dropout(0.15))
    # Más capas
    CNNmodel.add(Conv2D(96, (3, 3), activation='relu', padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(Conv2D(128, (3, 3), activation='relu',padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(MaxPooling2D((2, 2)))
    CNNmodel.add(Dropout(0.15))
    CNNmodel.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(Conv2D(256, (3, 3), activation='relu',padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(Conv2D(512, (3, 3), activation='relu',padding='same'))
    CNNmodel.add(BatchNormalization())
    CNNmodel.add(MaxPooling2D((2, 2)))
    CNNmodel.add(Dropout(0.15))
    CNNmodel.add(Flatten())
    CNNmodel.add(Dense(512, activation='relu'))
    CNNmodel.add(Dense(25, activation='softmax'))
    optimum= Adam(learning_rate=0.0001)

    # Compilar el modelo CNN con los parámetros definidos
    CNNmodel.compile(loss='categorical_crossentropy', optimizer=optimum, metrics=['accuracy',tf.keras.metrics.AUC(curve="ROC"),tf.keras.metrics.FalsePositives(),tf.keras.metrics.TruePositives(),tf.keras.metrics.FalseNegatives(),tf.keras.metrics.TrueNegatives(),tf.keras.metrics.SensitivityAtSpecificity(0.5)])

    return CNNmodel

In [8]:
# Utilizamos el KerasClassifier otra vez
model_stolen = KerasClassifier(
    model=create_blank_model(), 
    clip_values=(0, 255)
    )

In [12]:
# Extracting a thieved classifier
# by training the reference model
stolen_classifier = copycat_cnn.extract(
    x=train_generator, 
    #y=train_labels_stolen, 
    thieved_classifier=model_stolen
    )

AttributeError: 'DirectoryIterator' object has no attribute 'shape'